In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from mahts import HTSDistributor
from scipy.stats import iqr

import matplotlib.pyplot as plt

***

In [2]:
hierarchy = pd.read_parquet("../input/hierarchy_raw.parquet")

/home/turing/miniconda/lib/python3.7/site-packages/fastparquet/dataframe.py:5: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import CategoricalIndex, RangeIndex, Index, MultiIndex


In [3]:
infile = open("../input/encoders.pkl", "rb")
encoders = pickle.load(infile)
infile.close()

***
## Approach: level1 -> level2 -> level3 -> level12

In [4]:
!ls ../output/kaggle_submissions/

fnu050-num-leaves-76.csv	 m5-first-public-notebook-under-0-50.csv
m5-accuracy-poisson-tweedie.csv  m5-forecast-v2-python.csv
m5-accuracy-tweedie-is-back.csv  m5-forecaster-v2.csv


In [5]:
forecast_level12 = pd.read_csv("../output/lgbm_bottom_v48.csv.gz")
forecast_level12.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.810286,0.783921,0.723381,0.775287,0.942350,1.193541,0.871396,0.807392,0.766571,...,0.925544,1.163989,1.211029,0.913194,0.827492,0.768872,0.718190,0.840552,1.094288,1.014094
1,HOBBIES_1_002_CA_1_validation,0.151044,0.197842,0.213772,0.224499,0.261841,0.307496,0.433642,0.214342,0.206178,...,0.253056,0.311303,0.309737,0.206105,0.211625,0.210578,0.222463,0.267798,0.329565,0.332412
2,HOBBIES_1_003_CA_1_validation,0.483419,0.441670,0.461556,0.405231,0.619941,0.750705,0.484661,0.419990,0.430547,...,0.583329,0.664910,0.605112,0.377984,0.357978,0.349530,0.360897,0.514347,0.645241,0.625011
3,HOBBIES_1_004_CA_1_validation,1.408120,1.179424,1.390610,1.445403,1.879867,2.774164,3.851569,1.784356,1.451984,...,2.078965,2.899898,3.692053,1.971314,1.516034,1.525099,1.587655,2.104526,2.945701,3.360366
4,HOBBIES_1_005_CA_1_validation,1.069765,0.941750,0.975759,1.034548,1.283991,1.662992,1.409299,1.114342,1.161364,...,1.240064,1.575096,1.608372,1.135913,1.023454,1.044744,1.095417,1.315404,1.634598,1.661013


In [6]:
date_mapping1 = {f"F{i}": pd.to_datetime("2016-04-24")+pd.DateOffset(days=i) for i in range(1,29)}
date_mapping2 = {f"F{i}": pd.to_datetime("2016-05-22")+pd.DateOffset(days=i) for i in range(1,29)}

forecast_level12_valid = (
    forecast_level12
    .loc[:30489, :]
    .assign(id = lambda x: x.id.map(lambda x: x.replace("_validation","")))
    .rename(date_mapping1, axis=1)
    .set_index("id")
    .transpose()
)

forecast_level12_eval = (
    forecast_level12
    .loc[30490:, :]
    .assign(id = lambda x: x.id.map(lambda x: x.replace("_evaluation","")))
    .rename(date_mapping2, axis=1)
    .set_index("id")
    .transpose()
)

In [7]:
forecast_level12_valid.tail()

id,HOBBIES_1_001_CA_1,HOBBIES_1_002_CA_1,HOBBIES_1_003_CA_1,HOBBIES_1_004_CA_1,HOBBIES_1_005_CA_1,HOBBIES_1_006_CA_1,HOBBIES_1_007_CA_1,HOBBIES_1_008_CA_1,HOBBIES_1_009_CA_1,HOBBIES_1_010_CA_1,...,FOODS_3_818_WI_3,FOODS_3_819_WI_3,FOODS_3_820_WI_3,FOODS_3_821_WI_3,FOODS_3_822_WI_3,FOODS_3_823_WI_3,FOODS_3_824_WI_3,FOODS_3_825_WI_3,FOODS_3_826_WI_3,FOODS_3_827_WI_3
2016-05-18 00:00:00,0.768872,0.210578,0.349530,1.525099,1.044744,0.859421,0.301849,8.759270,0.885132,0.536644,...,1.952619,2.009027,1.599500,0.894726,1.930690,0.301863,0.281172,0.691304,0.873817,1.863316
2016-05-19 00:00:00,0.718190,0.222463,0.360897,1.587655,1.095417,0.830600,0.302735,8.339695,0.836911,0.534477,...,1.862883,1.849196,1.502884,0.897840,1.859493,0.307634,0.287514,0.663412,0.840773,1.677886
2016-05-20 00:00:00,0.840552,0.267798,0.514347,2.104526,1.315404,0.877789,0.300183,9.084737,1.006483,0.642632,...,2.039560,1.964561,1.774682,1.164573,2.360017,0.318233,0.329342,0.779736,0.923639,2.157849
2016-05-21 00:00:00,1.094288,0.329565,0.645241,2.945701,1.634598,1.072464,0.392394,10.691849,1.292462,0.756210,...,2.044512,2.322018,1.878656,1.287038,2.885314,0.390904,0.373473,0.866286,0.915628,2.392032
2016-05-22 00:00:00,1.014094,0.332412,0.625011,3.360366,1.661013,1.116498,0.376765,8.852718,1.364925,0.841295,...,2.063096,2.328230,1.760480,1.418262,2.576731,0.431168,0.396320,0.907803,1.005610,2.424909


In [8]:
forecast_level1_valid = (
    pd.read_csv("../output/forecast_level1.csv", parse_dates=["ds"])
    .query("ds <= '2016-05-22'")
    .set_index("ds")
    .rename({"y_pred":"root"}, axis=1)
)

forecast_level1_eval = (
    pd.read_csv("../output/forecast_level1.csv", parse_dates=["ds"])
    .query("ds >= '2016-05-23'")
    .set_index("ds")
    .rename({"y_pred":"root"}, axis=1)
)

In [9]:
forecast_level1_eval.tail()

,root
ds,
2016-06-15,41821.755327
2016-06-16,40034.043678
2016-06-17,43855.204804
2016-06-18,49876.376823
2016-06-19,47095.657592


In [10]:
forecast_level2_valid = (
    pd.read_csv("../output/forecast_level2.csv", parse_dates=["ds"])
    .query("ds <= '2016-05-22'")
    .assign(state_id = lambda x: encoders["state"].inverse_transform(x.state_id))
    .pivot(index="ds", columns="state_id", values="y_pred")
)

forecast_level2_eval = (
    pd.read_csv("../output/forecast_level2.csv", parse_dates=["ds"])
    .query("ds >= '2016-05-23'")
    .assign(state_id = lambda x: encoders["state"].inverse_transform(x.state_id))
    .pivot(index="ds", columns="state_id", values="y_pred")
)

In [11]:
forecast_level2_eval.head()

state_id,CA,TX,WI
ds,,,
2016-05-23,16436.528628,10305.473531,10386.920153
2016-05-24,14835.246738,9336.073186,9902.577344
2016-05-25,14536.126882,9350.385160,9849.840648
2016-05-26,14897.844553,9607.309100,10267.400059
2016-05-27,16542.566114,10359.831406,11796.342535


In [12]:
forecast_level3_valid = (
    pd.read_csv("../output/forecast_level3.csv", parse_dates=["ds"])
    .query("ds <= '2016-05-22'")
    .assign(store_id = lambda x: encoders["store"].inverse_transform(x.store_id))
    .pivot(index="ds", columns="store_id", values="y_pred")
)

forecast_level3_eval = (
    pd.read_csv("../output/forecast_level3.csv", parse_dates=["ds"])
    .query("ds >= '2016-05-23'")
    .assign(store_id = lambda x: encoders["store"].inverse_transform(x.store_id))
    .pivot(index="ds", columns="store_id", values="y_pred")
)

In [13]:
forecast_level3_valid.head()

store_id,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
ds,,,,,,,,,,
2016-04-25,4091.490967,3834.845789,5917.302713,2589.053279,2986.369223,3658.009162,3670.416690,3214.306487,4328.170338,3242.235332
2016-04-26,3683.369169,3671.675410,5486.626948,2432.476441,2765.738437,3334.837524,3393.874701,3106.612502,4167.746957,3026.673254
2016-04-27,3563.569356,3647.429944,5250.504504,2358.486993,2687.865582,3289.101873,3296.410209,3121.478780,4147.533751,2940.078248
2016-04-28,3603.053812,3689.192840,5180.914969,2359.378619,2659.252982,3297.943273,3280.910317,3181.940656,4173.130817,2980.661975
2016-04-29,4349.159399,4334.665510,5661.639214,2543.332982,2961.314859,3760.779512,3610.539906,3928.809860,4872.960884,3716.463090


***

In [14]:
hierarchy_dict = {"root":hierarchy.state_id.unique()}

for state_id in hierarchy.state_id.unique():
    hierarchy_dict[state_id] = hierarchy.query("state_id == @state_id").store_id.unique()
    
for store_id in hierarchy.store_id.unique():
    hierarchy_dict[store_id] = hierarchy.query("store_id == @store_id").id.unique()

In [15]:
hts = HTSDistributor(hierarchy_dict)

In [16]:
forecast_valid = pd.concat([forecast_level1_valid, forecast_level2_valid, forecast_level3_valid, forecast_level12_valid],
                           axis=1)

In [17]:
forecast_eval = pd.concat([forecast_level1_eval, forecast_level2_eval, forecast_level3_eval, forecast_level12_eval],
                           axis=1)

***
### weights

In [18]:
residuals_level2 = pd.read_parquet("../output/residuals_level2.parquet")
residuals_level2["state_id"] = encoders["state"].inverse_transform(residuals_level2.state_id)
residuals_level2

,state_id,residual
0,CA,401.273500
1,TX,379.039934
2,WI,405.244333


In [19]:
residuals_level3 = pd.read_parquet("../output/residuals_level3.parquet")
residuals_level3["store_id"] = encoders["store"].inverse_transform(residuals_level3.store_id)
residuals_level3

,store_id,residual
0,CA_1,191.696049
1,CA_2,186.930838
2,CA_3,243.934122
3,CA_4,113.910485
4,TX_1,156.807282
5,TX_2,220.916225
6,TX_3,165.716004
7,WI_1,205.164055
8,WI_2,204.697503
9,WI_3,200.869899


In [20]:
residuals_level12 = pd.read_parquet("../output/residuals_level12.parquet")
residuals_level12["item_id"] = encoders["item"].inverse_transform(residuals_level12.item_id)
residuals_level12["store_id"] = encoders["store"].inverse_transform(residuals_level12.store_id)
residuals_level12 = pd.merge(hierarchy.loc[:, ["store_id","item_id","id"]],
                             residuals_level12,
                             how="left", 
                             on=["item_id","store_id"])
# there are missing item_id,store_id in residuals dataframe
residuals_level12["residual"] = residuals_level12.residual.fillna(1)
residuals_level12

,store_id,item_id,id,residual
0,CA_1,HOBBIES_1_001,HOBBIES_1_001_CA_1,0.848878
1,CA_1,HOBBIES_1_002,HOBBIES_1_002_CA_1,0.586911
2,CA_1,HOBBIES_1_003,HOBBIES_1_003_CA_1,0.694128
3,CA_1,HOBBIES_1_004,HOBBIES_1_004_CA_1,2.009810
4,CA_1,HOBBIES_1_005,HOBBIES_1_005_CA_1,1.308492
...,...,...,...,...
30485,WI_3,FOODS_3_823,FOODS_3_823_WI_3,0.824179
30486,WI_3,FOODS_3_824,FOODS_3_824_WI_3,0.773221
30487,WI_3,FOODS_3_825,FOODS_3_825_WI_3,1.167359
30488,WI_3,FOODS_3_826,FOODS_3_826_WI_3,1.249926


In [21]:
#func = lambda x: np.log1p(x)
#func = lambda x: x**1
#func = lambda x: x*np.log1p(x)
func = lambda x: x**2

weights_level12 = {row["id"] : 1./func(row["residual"]) for _,row in residuals_level12.iterrows()}
weights_level3 = {row["store_id"] : 1./func(row["residual"]) for _,row in residuals_level3.iterrows()}
weights_level2 = {row["state_id"] : 1./func(row["residual"]) for _,row in residuals_level2.iterrows()}
weights_level1 = {"root" : 1./func(882.9388819910898)}

In [22]:
weights = {**weights_level1, **weights_level2, **weights_level3, **weights_level12}

***

In [23]:
fcst_valid = hts.compute_optimal_combination(forecast_valid, weights=weights)
fcst_valid.head()

,root,CA,TX,WI,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,...,FOODS_3_818_WI_3,FOODS_3_819_WI_3,FOODS_3_820_WI_3,FOODS_3_821_WI_3,FOODS_3_822_WI_3,FOODS_3_823_WI_3,FOODS_3_824_WI_3,FOODS_3_825_WI_3,FOODS_3_826_WI_3,FOODS_3_827_WI_3
0,36893.409231,16376.077516,10011.300711,10506.031003,4138.770823,3737.879486,5785.306924,2714.120283,3031.117537,3478.387685,...,1.822008,1.762734,1.538116,0.528997,1.956369,0.269357,0.246194,0.660079,1.002995,0.271153
1,36037.336955,15728.578406,9810.261283,10498.497267,3878.620488,3726.571771,5500.388027,2622.998119,2992.255359,3393.418216,...,1.918417,1.587466,1.549626,0.678734,1.742851,0.283815,0.201118,0.547456,0.992088,1.245425
2,35283.731603,15314.862415,9579.256467,10389.612720,3803.803368,3668.386306,5286.788031,2555.884711,2904.032244,3357.466953,...,1.641522,1.432039,1.437937,0.728419,1.625235,0.291830,0.192581,0.539918,0.862743,1.597456
3,35752.018059,15516.466422,9587.539463,10648.012174,3898.643182,3773.374827,5265.063490,2579.384922,2882.507071,3397.315117,...,1.560274,1.532033,1.528444,0.827741,1.667247,0.286598,0.203468,0.552979,0.850095,1.785568
4,42039.996520,17932.319890,10805.768136,13301.908494,4708.735468,4611.240640,5802.442913,2809.900869,3240.379387,3847.716903,...,1.669204,1.791314,1.779189,1.137220,2.233493,0.345367,0.234224,0.607663,0.881192,1.689620


In [24]:
fcst_eval = hts.compute_optimal_combination(forecast_eval, weights=weights)
fcst_eval.head()

,root,CA,TX,WI,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,...,FOODS_3_818_WI_3,FOODS_3_819_WI_3,FOODS_3_820_WI_3,FOODS_3_821_WI_3,FOODS_3_822_WI_3,FOODS_3_823_WI_3,FOODS_3_824_WI_3,FOODS_3_825_WI_3,FOODS_3_826_WI_3,FOODS_3_827_WI_3
0,39514.184757,17728.348091,10828.212001,10957.624665,4566.439158,3977.333392,6257.043595,2927.531946,3255.328593,3807.952028,...,1.952777,2.231045,1.344554,0.763776,1.849440,0.310264,0.325530,0.694421,0.815505,1.938562
1,35564.457781,15767.096298,9312.264410,10485.097073,4020.464359,3669.955635,5575.174577,2501.501728,2916.148577,3047.125832,...,1.892684,1.740524,1.454573,0.849227,1.619353,0.288708,0.266896,0.565403,0.792076,1.846500
2,36515.599218,15908.479786,9929.393414,10677.726018,4013.835069,3778.520788,5475.711382,2640.412547,2979.565561,3618.944150,...,1.792983,2.013160,1.451055,0.908723,1.693291,0.272746,0.274278,0.595844,0.798911,1.746775
3,36639.906840,16042.506884,9754.844590,10842.555366,4060.773297,3927.992840,5443.919295,2609.821453,2955.689051,3475.219371,...,1.757839,1.920449,1.430307,0.934015,1.695796,0.266796,0.277245,0.573120,0.785968,1.513279
4,41678.188669,18140.413857,10822.638158,12715.136654,4742.587105,4704.665521,5874.695119,2818.466112,3277.234223,3916.633061,...,1.866307,2.127861,1.724045,1.174061,2.100433,0.301147,0.309560,0.640188,0.830301,2.038678


***

In [25]:
fcst_valid.set_index(forecast_valid.index, inplace=True)
fcst_eval.set_index(forecast_eval.index, inplace=True)

fcst_valid = fcst_valid.loc[:, hts.bottom_nodes].transpose()
fcst_eval = fcst_eval.loc[:, hts.bottom_nodes].transpose()

fcst_valid.columns = [f"F{i}" for i in range(1,29)]
fcst_eval.columns = [f"F{i}" for i in range(1,29)]

fcst_valid = (fcst_valid
              .reset_index()
              .rename({"index":"id"}, axis=1))

fcst_eval= (fcst_eval
              .reset_index()
              .rename({"index":"id"}, axis=1))

fcst_valid["id"] = fcst_valid.id.apply(lambda x: x+"_validation")
fcst_eval["id"] = fcst_eval.id.apply(lambda x: x+"_evaluation")

***
### submission

In [26]:
submission = pd.read_csv("../input/sample_submission.csv")

In [27]:
all_predictions = pd.concat([fcst_valid, fcst_eval], ignore_index=True)

In [28]:
assert set(all_predictions.id) == set(submission.id), \
    "there are missing time series predictions."

In [29]:
submission = pd.merge(submission.loc[:, ["id"]], all_predictions)

In [30]:
!ls ../output

forecast_level1.csv	lgbm_bottom_v23.csv.gz	lgbm_bottom_v41.csv.gz
forecast_level2.csv	lgbm_bottom_v24.csv.gz	lgbm_bottom_v42.csv.gz
forecast_level3.csv	lgbm_bottom_v25.csv.gz	lgbm_bottom_v43.csv.gz
forecast_root.csv	lgbm_bottom_v26.csv.gz	lgbm_bottom_v44.csv.gz
kaggle_submissions	lgbm_bottom_v27.csv.gz	lgbm_bottom_v45.csv.gz
lgbm_bottom_v1.csv.gz	lgbm_bottom_v28.csv.gz	lgbm_bottom_v46.csv.gz
lgbm_bottom_v10.csv.gz	lgbm_bottom_v29.csv.gz	lgbm_bottom_v47.csv.gz
lgbm_bottom_v11.csv.gz	lgbm_bottom_v3.csv.gz	lgbm_bottom_v48.csv.gz
lgbm_bottom_v12.csv.gz	lgbm_bottom_v30.csv.gz	lgbm_bottom_v49.csv.gz
lgbm_bottom_v13.csv.gz	lgbm_bottom_v31.csv.gz	lgbm_bottom_v5.csv.gz
lgbm_bottom_v14.csv.gz	lgbm_bottom_v32.csv.gz	lgbm_bottom_v50.csv.gz
lgbm_bottom_v15.csv.gz	lgbm_bottom_v33.csv.gz	lgbm_bottom_v52.csv.gz
lgbm_bottom_v16.csv.gz	lgbm_bottom_v34.csv.gz	lgbm_bottom_v6.csv.gz
lgbm_bottom_v17.csv.gz	lgbm_bottom_v35.csv.gz	lgbm_bottom_v7.csv.gz
lgbm_bottom_v18.csv.gz	lgbm_bottom_v36.csv.gz	lgbm_bottom

In [31]:
file_path = "../output/lgbm_bottom_v51.csv.gz"

if os.path.isfile(file_path):
    print("File already exists.")
else:
    submission.to_csv(file_path, index=False, compression="gzip")

In [32]:
!kaggle competitions submit -c m5-forecasting-accuracy -f ../output/lgbm_bottom_v51.csv.gz -m ""

100%|██████████████████████████████████████| 14.7M/14.7M [00:04<00:00, 3.73MB/s]
Successfully submitted to M5 Forecasting - Accuracy

***

In [33]:
# difference between base submission and new submission
# func = lambda x: x**1
submission
base_submission = pd.merge(submission.loc[:, ["id"]], forecast_level12, how="left", on="id")
diff = np.sqrt(np.mean((submission.loc[:, [f"F{i}" for i in range(1,29)]].values - base_submission.loc[:, [f"F{i}" for i in range(1,29)]].values)**2))
print(diff)

0.5824470928084458


In [35]:
# difference between base submission and new submission
# func = lambda x: x**2
submission
base_submission = pd.merge(submission.loc[:, ["id"]], forecast_level12, how="left", on="id")
diff = np.sqrt(np.mean((submission.loc[:, [f"F{i}" for i in range(1,29)]].values - base_submission.loc[:, [f"F{i}" for i in range(1,29)]].values)**2))
print(diff)

1.211776998514943


***

In [33]:
# difference between base submission and new submission
# func = lambda x: np.log1p(x)
submission
base_submission = pd.merge(submission.loc[:, ["id"]], forecast_level12, how="left", on="id")
diff = np.sqrt(np.mean((submission.loc[:, [f"F{i}" for i in range(1,29)]].values - base_submission.loc[:, [f"F{i}" for i in range(1,29)]].values)**2))
print(diff)

0.6911525385859656


In [49]:
# difference between base submission and new submission
# func = lambda x: x**1
submission
base_submission = pd.merge(submission.loc[:, ["id"]], forecast_level12, how="left", on="id")
diff = np.sqrt(np.mean((submission.loc[:, [f"F{i}" for i in range(1,29)]].values - base_submission.loc[:, [f"F{i}" for i in range(1,29)]].values)**2))
print(diff)

0.852040145705367


In [62]:
# difference between base submission and new submission
# func = lambda x: x*np.log1p(x)
submission
base_submission = pd.merge(submission.loc[:, ["id"]], forecast_level12, how="left", on="id")
diff = np.sqrt(np.mean((submission.loc[:, [f"F{i}" for i in range(1,29)]].values - base_submission.loc[:, [f"F{i}" for i in range(1,29)]].values)**2))
print(diff)

0.38115852204637


In [75]:
# difference between base submission and new submission
# func = lambda x: x**2
submission
base_submission = pd.merge(submission.loc[:, ["id"]], forecast_level12, how="left", on="id")
diff = np.sqrt(np.mean((submission.loc[:, [f"F{i}" for i in range(1,29)]].values - base_submission.loc[:, [f"F{i}" for i in range(1,29)]].values)**2))
print(diff)

0.30197726189362306


***